In [0]:
!git clone https://github.com/satproject/neuralheuristicsforsat.git
%cd neuralheuristicsforsat

In [0]:
%cd ..

In [0]:
!pip install -r requirements.in 

In [0]:
%tensorflow_version 1.12

In [0]:
!pip install torch
!pip install numpy

In [0]:
%cd neuralheuristicsforsat/

In [0]:
def make_example(inputs, sat):
    example = tf.train.Example(
        features=tf.train.Features(feature={
            "inputs": tf.train.Feature(
                float_list=tf.train.FloatList(value=list(inputs.flatten()))),
            "sat": tf.train.Feature(
                float_list=tf.train.FloatList(value=list(sat.flatten())))
        })
    )
    return example.SerializeToString()


def tf_serialize_example(sample):
    tf_string = tf.py_func(make_example, (sample["inputs"], sample["sat"]), tf.string)
    return tf.reshape(tf_string, ())

In [0]:
import tensorflow as tf
import numpy as np

import cnf_dataset
from tqdm import tqdm
import argparse
import os
import random
#Reimport a module in python while interactive
from importlib import reload
reload(cnf_dataset)
reload(tf)


def main_generate(observations, complexity, job):

    random.seed(int(job))
    print("Set random seed to {}".format(int(job)))

    dirname = "sr_{}".format(complexity)  
    filename = "train_{}_sr_{}.tfrecord".format(job, complexity)
    options = {
        "PROCESSOR_NUM": 24,
        "CLAUSE_NUM": 10*complexity,
        "VARIABLE_NUM": complexity,
        "MIN_VARIABLE_NUM": 1,
        "BATCH_SIZE": 1,
        "CLAUSE_SIZE": 2,
        "MIN_CLAUSE_NUM": 2,
        "SR_GENERATOR": False
    }
    n_observations = observations

    if not os.path.exists(dirname):
        os.makedirs(dirname)
        print("Created directory {}".format(dirname))

    with cnf_dataset.PoolDatasetGenerator(options) as generator, \
            tf.python_io.TFRecordWriter(os.path.join(dirname,filename)) as writer:

        for _ in tqdm(range(n_observations)):
            sample_with_labels = generator.generate_batch(representation='cnfs')
            print(sample_with_labels.inputs.astype(np.float32))
            tf_sample = {
                 "inputs": np.squeeze(sample_with_labels.inputs.astype(np.float32), 0),
                 "sat": np.squeeze(np.asarray(sample_with_labels.sat_labels).astype(np.float32), 0)
            }
            
            serialized = make_example(**tf_sample)
            writer.write(serialized)
main_generate(10, 10 ,12312)

In [0]:
def accuracy(x, y):
  return sum(1 for a,b in zip(x,y) if a == b) / len(x)


In [0]:
import itertools
import os
import tensorflow as tf


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(list(map(int, seq[int(last):int(last + avg)])))
        last += avg

    return out

def main():
    n = 100
    tfrecord_location = '/content/neuralheuristicsforsat/sr_100'
    name = "train_21021_sr_100.tfrecord"
    filename = os.path.join(tfrecord_location, name)
    #filename = "train_21021_sr_10.tfrecord"

    record_iterator = tf.python_io.tf_record_iterator(path=filename)
    preds = []
    targes = []
    batch_size = n

    for string_record in itertools.islice(record_iterator, 100):
        example = tf.train.Example()
        example.ParseFromString(string_record)
        

        M = len(example.features.feature["inputs"].float_list.value)//2
        inputs = chunkIt(example.features.feature["inputs"].float_list.value, M)

        preds.append(predict({'n': n, 'm': M}, inputs))
        targes.append(int(example.features.feature["sat"].float_list.value[0]))

    print(accuracy(preds, targes))

if __name__ == '__main__':
    main()

